In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [87]:
DATA_FOLDER = '../data/'
DATA_ZIP = DATA_FOLDER + 'datasets.zip'

DATA_TRAIN_PATH = DATA_FOLDER + 'train.csv'
DATA_TEST_PATH = DATA_FOLDER + 'test.csv' 

## Load the training data into feature matrix, class labels, and event ids:

In [80]:
from zipfile import ZipFile
import os.path

#Uncompress zip if a file is missing
if not (os.path.isfile(DATA_TRAIN_PATH) and os.path.isfile(DATA_TEST_PATH)):
    with ZipFile(DATA_ZIP, 'r') as zip: 
        zip.printdir()
        print("Extracting datasets.zip ...")
        zip.extractall(DATA_FOLDER)
        print("Extraction done!")


In [94]:
from proj1_helpers import *
from split_data import *

y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

split_ratio = 0.8 
tX_train, tX_validation, y_train, y_validation = split_data(tX, y, split_ratio)

In [99]:
row_format = "{:<15}" * 3
print(row_format.format("", "Training", "Validation"))
print(row_format.format("Features", str(tX_train.shape), str(tX_validation.shape)))
print(row_format.format("Labels", str(y_train.shape), str(y_validation.shape)))

               Training       Validation     
Features       (200000, 30)   (50000, 30)    
Labels         (200000,)      (50000,)       


## Do your thing crazy machine learning thing here :) ...

In [84]:
from least_squares import *
from costs import *

weights = least_squares(y, tX)

print(compute_loss(y_train, tX_train, w))
print(compute_loss(y_validation, tX_validation, w))

0.33943505815803526
0.34069381475303295


## Generate predictions and save ouput in csv format for submission:

In [81]:
DATA_TEST_PATH = DATA_FOLDER + 'test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [86]:
OUTPUT_PATH = DATA_FOLDER + 'submission.csv' 
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)